<div class="alert alert-block alert-info">  
    <h1><strong>👨‍💻 Getting Started with SIIM-FISABIO-RSNA COVID-19 Detection</strong></h1>
    <i></i>
</div>

<center><img src="https://images.livemint.com/img/2020/12/09/600x338/AFP_8VR7GD_1606728343841_1606728388950_1607479144566.jpg"></center>

<div class="alert alert-block alert-info">  
<h1><strong>Introduction</strong></h1>
    <p>At the end of 2019, the coronavirus disease aroused in “Wuhan”, the city of china. Since January 2020, It has been extensively increased all around the world. On 30 January 2020, the WHO announced public health emergency internationally because of COVID-19. The disease was named coronavirus disease (COVID-19) by WHO in February 2020. The epidemic COVID-19 disease is caused by a virus called "Severe Acute Respiratory Syndrome" coronavirus-2 (SARS-CoV-2). The coronavirus is a family of several viruses that causes disease like "Severe Acute Respiratory Syndrome" (SARS-CoV) and "Middle East Respiratory Syndrome". Nowadays, countries like the USA, India, Brazil, and Russia struggling with the COVID-19 as it is the cause of the increase in the death rate. It can be spread to humans as well as other species of animals such as cats, cattle, bats, and camels. It is can be deadly for people with a weakened immune system. Physicians and specialists of infectious diseases around the world are starving hard to discover a treatment for the disease. The diseases can be spreading by touching and sitting or standing close to infected person. So, the COVID-19 infected person should to be identified and isolated before he might infect others. The earlier stage symptoms of COVID-19 include “cough”, “fatigue”, “fever”, and “myalgia”. The more intense situation of disease includes the heart damage, the respiratory problem, and internal infections. Some other abnormal states are observed in Covid-19 patients Chest CT and X-ray images. In most cases, the doctors use the patient’s X-ray and CT scan images of the patient’s chest for earlier stage diagnose of COVID-19. Chest radiography is the most widely used method of pneumonia diagnosis worldwide. It is a faster and cheaper clinical method of diagnosis. It is preferred over computed tomography (CT) and MRI because of lower radiation dose, easy availability, and cost. So, X-ray is the first choice of the radiologist for chest pathology detection and it is also applied for confirmation of COVID-19 patients. Therefore, this study focus on the use of X-ray images for COVID-19 detection in patients. </p>
    <br>
        <hr>
      <b>Problem description/definition: </b> 
    <hr>
<ul>
   This notebook will develop a deep neural network and machine learning based X-ray image classification system for identifying the patient with coronavirus so clinical staff can easily and efficiently differentiate the patient with coronavirus from the patient who don’t have the disease. The proposed system can save both time and human resource.
</ul>


<hr>
<b>Evolution measures: </b> 
<hr>
<ul>
<p> After training the model, we apply the evaluation measures to check that how the model is getting predictions. We will use the following evaluation measures to evaluate the performance of the models:</p>
    <li>Accuracy</li>
    <li>Confusion matrix with plot</li>
    <li>ROC Curve</li>
</ul>
<hr>

 
<hr>
<b>Source of Data: </b> 
<hr> 
 <a href="https://www.kaggle.com/c/siim-covid19-detection">https://www.kaggle.com/c/siim-covid19-detection</a>
   
</div>

<div class="alert alert-block alert-success">  
    <h1><strong>Importing Python Libraries 📕 📗 📘 📙</strong></h1>
    <i></i>
</div>

In [ ]:
import os
import seaborn as sns
from pandas_profiling import ProfileReport
import pydicom
from glob import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
from skimage import exposure
import cv2
import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-danger">  
    <h1><strong>Sample Images</strong></h1>
    <i></i>
</div>

In [ ]:
dataset_dir = '../input/siim-covid19-detection'
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data
        
    
def plot_img(img, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

def plot_imgs(imgs, cols=5, size=7, is_rgb=True, title="", cmap='gray', img_size=(500,500)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()
dicom_paths = glob(f'{dataset_dir}/train/*/*/*.dcm')
imgs = [dicom2array(path) for path in dicom_paths[:20]]
plot_imgs(imgs)

<div class="alert alert-block alert-info">  
    <h2><strong>The above sample Images after applying preprocessing Technique which is  equalize histogram</strong></h2>
    <i></i>
</div>

In [ ]:
imgs = [exposure.equalize_hist(img) for img in imgs]
plot_imgs(imgs)

If we foucs deeply on the last image, it was not clear before applying the equalize histogram but after that it is clear and give the meaningful information.

<div class="alert alert-block alert-danger">  
    <h1><strong>Loading data</strong></h1>
    <i></i>
</div>

In [ ]:
train = pd.read_csv(f'{dataset_dir}/train_image_level.csv')
train_study = pd.read_csv(f'{dataset_dir}/train_study_level.csv')

<div class="alert alert-block alert-success">  
    <h1><strong>Exploratory data analysis</strong></h1>
    <i></i>
</div>

<div class="alert alert-block alert-info">  
    <h4><strong>Train</strong></h4>
    <i></i>
</div>

In [ ]:
# Five top records of data

train.head()

In [ ]:
# Five last records of data

train.tail()

In [ ]:
# Coloumns/features in data

train.columns

In [ ]:
# Length of data

print('lenght of data is', len(train))

In [ ]:
# Shape of data

train.shape

In [ ]:
# Data information

train.info()

In [ ]:
# Data types of all coloumns

train.dtypes

In [ ]:
# Checking missing Values

train[train.isnull().any(axis=1)].head()

In [ ]:
# Count of missing values

np.sum(train.isnull().any(axis=1))

In [ ]:
# Counts of missing values in each column

train.isnull().sum()

<div class="alert alert-block alert-danger">  
    <h3><strong>Data Visualization of train</strong></h3>
    <i></i>
</div>

In [ ]:
train_vis = ProfileReport(train, title="train_image_level_report")
train_vis

<div class="alert alert-block alert-info">  
    <h4><strong>train_study</strong></h4>
    <i></i>
</div>

In [ ]:
# Five top records of data

train_study.head()

In [ ]:
# Five last records of data

train_study.tail()

<div class="alert alert-block alert-danger">  
    <h3><strong>Data Visualization of train study</strong></h3>
    <i></i>
</div>

In [ ]:
train_st = ProfileReport(train_study, title="train study of images")
train_st

# Merging train study in train

In [ ]:
train_study['StudyInstanceUID'] = train_study['id'].apply(lambda x: x.replace('_study', ''))
del train_study['id']
train = train.merge(train_study, on='StudyInstanceUID')
train.head()
def bar_plot(train_df, variable):
    var = train_df[variable]
    varValue = var.value_counts()
    
    # visualize
    plt.figure(figsize = (9,3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}: \n {}".format(variable,varValue))
    
train['target'] = 'Negative for Pneumonia'
train.loc[train['Typical Appearance']==1, 'target'] = 'Typical Appearance'
train.loc[train['Indeterminate Appearance']==1, 'target'] = 'Indeterminate Appearance'
train.loc[train['Atypical Appearance']==1, 'target'] = 'Atypical Appearance'
bar_plot(train, 'target') 

In [ ]:
train["target"].value_counts().head(7).plot(kind = 'pie', autopct='%1.1f%%', figsize=(8, 8)).legend()

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(20,16))
sns.kdeplot(train["Negative for Pneumonia"], shade=True,ax=ax[0,0],color="#ffb4a2")
ax[0,0].set_title("Negative for Pneumonia Distribution",font="Serif")
sns.kdeplot(train["Typical Appearance"], shade=True,ax=ax[0,1],color="#e5989b")
ax[0,1].set_title("Typical Appearance Distribution",font="Serif")
sns.kdeplot(train["Indeterminate Appearance"], shade=True,ax=ax[1,0],color="#b5838d")
ax[1,0].set_title("Indeterminate Appearance Distribution",font="Serif")
sns.kdeplot(train["Atypical Appearance"], shade=True,ax=ax[1,1],color="#6d6875")
ax[1,1].set_title("Atypical Appearance Distribution",font="Serif")
plt.show()

<div class="alert alert-block alert-danger">  
    <h3><strong>Localization of Labels with Bounding Boxes</strong></h3>
    <i></i>
</div>

In [ ]:
#train = train[~train.boxes.isnull()] 
class_names = ['Typical Appearance','Negative for Pneumonia', 'Indeterminate Appearance', 'Atypical Appearance']
unique_classes = np.unique(train[class_names].values, axis=0)
imgs = []
label2color = {
    '[1, 0, 0, 0]': [255,0,0], # Typical Appearance
    '[0, 1, 0, 0]': [0,255,0], # Indeterminate Appearance
    '[0, 0, 1, 0]': [0,0,255], # Atypical Appearance
    '[0, 0, 0, 1]': [255,255,0], # Negative for Pneumonia
}
print('Typical Appearance: red')
print('Indeterminate Appearance: green')
print('Atypical Appearance: blue')
print('Negative for Pneumonia: yellow')
print(' \n  \n ')
thickness = 4
scale = 5
for _, row in train[train['Negative for Pneumonia']==0].iloc[:20].iterrows():
    study_id = row['StudyInstanceUID']
    img_path = glob(f'{dataset_dir}/train/{study_id}/*/*')[0]
    img = dicom2array(path=img_path)
    img = cv2.resize(img, None, fx=1/scale, fy=1/scale)
    img = np.stack([img, img, img], axis=-1)
    
    claz = row[class_names].values
    color = label2color[str(claz.tolist())]

    bboxes = []
    bbox = []
    for i, l in enumerate(row['label'].split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l)/scale)
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []    
    
    for box in bboxes:
        img = cv2.rectangle(
            img,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, thickness
    )
    img = cv2.resize(img, (500,500))
    imgs.append(img)
    
plot_imgs(imgs, cmap=None)

> **Model Training with Transfer learning, coming Soon**

<center><img src="https://thumbs.gfycat.com/AshamedWeightyDachshund-max-1mb.gif"></center>

<div class="alert alert-block alert-info">  
    <h3><strong>Guideline to submit the results</strong></h3>
    <i></i>
</div>

> **Check the sample_submission.csv file so we will know the format that how can submit the predictions. Before we do so, it's worth reminding ourselves that this is a code-only competition, meaning that your submission file has to be generated in a script/notebook. The sample_submission.csv file demonstrated what kind of file needs to be produced:**

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list
study_len = df[df['id_last_str'] == 'y'].shape[0]
study_str = 'negative 1 0 0 1 1 atypical 1 0 0 1 1 typical 1 0 0 1 1 indeterminate 1 0 0 1 1'
for i in range(study_len):
    df.loc[i, 'PredictionString'] = study_str
submission_file = df[['id', 'PredictionString']]

In [ ]:
submission_file.head(10)

> **The submission file is look like above. We also have to provide the image-level bounding box. These will be of the format [class ID] [confidence score] [bounding box] as described earlier.**

In [ ]:
submission_file.to_csv('submission.csv', index=False)

> **We submitted the predictions but we will come back soon with model training and a lot more.**

<div class="alert alert-block alert-success">  
<h1>If you like my work, please upvote ^ 👍 my code so that i will be motivated to share more valuable and helpful work on this competition 😍</h1>
        </p>
</div>